In [1]:
import sys
sys.path.extend(['..', '../', '../..', '../../' ])
%load_ext autoreload
%autoreload 2

# LeNet5 - MNIST

In [2]:
import paths
import keras
from src.parameters import metrics
from src.datasets.dataset_generators import DatasetGenerators
from src.keras_callback.time_history import TimeHistory
from src.export.csv_export import export_learning_history_to_csv, export_learning_validation_result_to_csv
from src.export.model_export import clear_exported_models
from src.keras_model.lenet5_network import LeNet5Network

Using TensorFlow backend.


In [3]:
network_name = 'lenet5'
fit_epochs = 50
dataset = 'mnist'
dataset_percentage = '5'

In [4]:
network = LeNet5Network()

In [5]:
dataset_generators = DatasetGenerators(
    name=f'{dataset}_{dataset_percentage}',
    dataset_dir=paths.data_location,
    image_size=network.image_size,
    batch_size=network.batch_size,
    grayscale=network.grayscale
)

Loading train set for mnist_5...
Found 2419 images belonging to 10 classes.
Loading test set for mnist_5...
Found 695 images belonging to 10 classes.
Loading validation set for mnist_5...
Found 349 images belonging to 10 classes.


In [6]:
model = network.get_compiled_model(
    classes=dataset_generators.test.num_classes
)

In [7]:
checkpoint_path = f'{paths.models_location}{network_name}/{dataset}/'
best_model_path = f'{checkpoint_path}best.h5'
clear_exported_models(checkpoint_path)

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=f'{checkpoint_path}{{epoch:02d}}-{{val_accuracy}}.h5',
    save_best_only=False,
    save_weights_only=False
)

best_callback = keras.callbacks.ModelCheckpoint(
    filepath=best_model_path,
    save_best_only=True,
    verbose=1,
    save_weights_only=False
)

time_callback = TimeHistory()

model_callbacks = [checkpoint_callback, best_callback, time_callback]

In [8]:
fit_result = model.fit_generator(
    generator=dataset_generators.train,
    epochs=fit_epochs,
    validation_data=dataset_generators.test,
    callbacks = model_callbacks
)

Epoch 1/50
76/76 [==============================] - 1s 12ms/step - loss: 2.0939 - accuracy: 0.6838 - precision: 0.8870 - recall: 0.8424 - val_loss: 0.3801 - val_accuracy: 0.8777 - val_precision: 0.9886 - val_recall: 0.9583

Epoch 00001: val_loss improved from inf to 0.38015, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/mnist/best.h5
Epoch 2/50
76/76 [==============================] - 1s 10ms/step - loss: 0.3091 - accuracy: 0.9115 - precision: 0.9338 - recall: 0.9236 - val_loss: 0.1315 - val_accuracy: 0.9151 - val_precision: 0.9167 - val_recall: 0.9242

Epoch 00002: val_loss improved from 0.38015 to 0.13153, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/mnist/best.h5
Epoch 3/50
76/76 [==============================] - 1s 10ms/step - loss: 0.1868 - accuracy: 0.9409 - precision: 0.9612 - recall: 0.9561 - val_loss: 0.1370 - val_accuracy: 0.9122 - val_precision: 0.9061 - val_recall: 0.9303

Epoch 00003: val_loss did not improve 


Epoch 00028: val_loss improved from 0.00336 to 0.00073, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/mnist/best.h5
Epoch 29/50
76/76 [==============================] - 1s 10ms/step - loss: 1.5362e-04 - accuracy: 1.0000 - precision: 0.9868 - recall: 0.9868 - val_loss: 5.2510e-04 - val_accuracy: 0.9612 - val_precision: 1.0000 - val_recall: 0.9935

Epoch 00029: val_loss improved from 0.00073 to 0.00053, saving model to /Users/igor/Python/CNN-Architecture-Comparison/models/lenet5/mnist/best.h5
Epoch 30/50
76/76 [==============================] - 1s 10ms/step - loss: 1.4242e-04 - accuracy: 1.0000 - precision: 0.9605 - recall: 0.9605 - val_loss: 0.0369 - val_accuracy: 0.9640 - val_precision: 1.0000 - val_recall: 0.9886

Epoch 00030: val_loss did not improve from 0.00053
Epoch 31/50
76/76 [==============================] - 1s 10ms/step - loss: 1.3162e-04 - accuracy: 1.0000 - precision: 0.9868 - recall: 0.9868 - val_loss: 0.6637 - val_accuracy: 0.9640 - val_pre

In [9]:
best_model = keras.models.load_model(best_model_path,
    custom_objects= {
        'precision': metrics.precision,
        'recall': metrics.recall
    }
)
validation_result = best_model.evaluate_generator(
    generator=dataset_generators.validate
)

## Results

### Epoch CSV

In [10]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-history.csv'
export_learning_history_to_csv(csv_file_path, fit_result, time_callback.times)

epoch, time, val_loss, val_accuracy, val_precision, val_recall
0,0,0,0,0,0
1,0.9289579391479492,0.3801450729370117,0.8776978254318237,0.9886363744735718,0.9583333730697632
2,0.8051037788391113,0.13152512907981873,0.9151079058647156,0.9166666269302368,0.9242424368858337
3,0.7669229507446289,0.13702434301376343,0.9122301936149597,0.9061327576637268,0.9303030371665955
4,0.796252965927124,0.05501749366521835,0.935251772403717,0.8863636255264282,0.924242377281189
5,0.786081075668335,0.1864074170589447,0.9410071969032288,0.9628787636756897,0.9757575988769531
6,0.7813239097595215,0.21887293457984924,0.9553956985473633,1.0,0.972727358341217
7,0.7805981636047363,0.18376226723194122,0.9381294846534729,0.9680736064910889,0.9935064911842346
8,0.7854580879211426,0.06466849148273468,0.936690628528595,0.9204545617103577,0.9343433976173401
9,0.7760889530181885,0.5888987183570862,0.9496402740478516,0.8681817650794983,0.8950216174125671
10,0.8324379920959473,0.38724324107170105,0.9438849091529846,0.9545

### Validation result

In [11]:
csv_file_path = f'{paths.results_location}{network_name}/{dataset}-result.csv'
export_learning_validation_result_to_csv(csv_file_path, validation_result, time_callback.times)

avg_epoch_time, time, val_loss, val_accuracy, val_precision, val_recall
0.806595368385315,40.32976841926575,0.026978904381394386,0.9512894153594971,0.9363636374473572,0.9818181991577148
